In [33]:
import pandas as pd
import numpy as np

In [34]:
df1 = pd.read_csv('subtaskA_train.csv').drop(columns=['Id']).rename(columns={'conspiratorial': 'Annotator_1'})
df2 = pd.read_csv('subtaskA_train.csv').drop(columns=['Id']).rename(columns={'conspiratorial': 'Annotator_2'})
df3 = pd.read_csv('subtaskA_train.csv').drop(columns=['Id']).rename(columns={'conspiratorial': 'Annotator_3'})

In [35]:
df1.head()

,comment_text,Annotator_1
0,⚡Se non ci fossero soldati non ci sarebbero gu...,0
1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1
2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1
3,L'Aspirina non aumenta la sopravvivenza dei pa...,0
4,L'Italia non puo' dare armi lo vieta la Costit...,0


In [36]:
df = df1.copy()
df['Annotator_2'] = df2['Annotator_2']
df['Annotator_3'] = df3['Annotator_3']

In [37]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,⚡Se non ci fossero soldati non ci sarebbero gu...,0,0,0
1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1,1,1
2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1,1,1
3,L'Aspirina non aumenta la sopravvivenza dei pa...,0,0,0
4,L'Italia non puo' dare armi lo vieta la Costit...,0,0,0


In [38]:
df.shape

(1842, 4)

random perturbation

In [39]:
for i in range(151):
  df.loc[np.random.randint(10), 'Annotator_2'] = np.random.randint(2)
  df.loc[np.random.randint(10), 'Annotator_3'] = np.random.randint(2)


In [40]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,⚡Se non ci fossero soldati non ci sarebbero gu...,0,1,1
1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1,1,1
2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1,0,1
3,L'Aspirina non aumenta la sopravvivenza dei pa...,0,1,0
4,L'Italia non puo' dare armi lo vieta la Costit...,0,1,1


# Cleaning

In [41]:
# random perturbation
for i in range(11):
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_1'] = np.random.randint(2,3)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_2'] = np.random.randint(2,3)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_3'] = np.random.randint(2,3)

In [42]:
def check_vals(unique_vals_exp, df, columns, return_dict=False):
  idxs = dict()
  for col in df.columns[1:]:
    unique_vals = np.unique(df[col])
    if not np.array_equal(unique_vals, unique_vals_exp):
      print('Errors found in col', col)
      print('Indexes of errors:', list(df.query(f'{col} not in {unique_vals_exp}').index))
      idxs[col] = list(df.query(f'{col} not in {unique_vals_exp}').index)

  if len(idxs) == 0:
      print('no errors found')

  if return_dict:
    return idxs


In [43]:
unique_vals_exp = [0,1]
check_vals(unique_vals_exp, df, df.columns[1:])

Errors found in col Annotator_1
Indexes of errors: [131, 239, 329, 578, 712, 918, 1161, 1359, 1466, 1753, 1828]
Errors found in col Annotator_2
Indexes of errors: [73, 99, 150, 446, 534, 650, 678, 1172, 1426, 1597, 1727]
Errors found in col Annotator_3
Indexes of errors: [194, 434, 635, 653, 726, 729, 962, 1106, 1254, 1536, 1669]


In [44]:
idx_dict = check_vals(unique_vals_exp, df, df.columns[1:], return_dict=True)
for col, idx in idx_dict.items():
  df.loc[idx, col] = 1

Errors found in col Annotator_1
Indexes of errors: [131, 239, 329, 578, 712, 918, 1161, 1359, 1466, 1753, 1828]
Errors found in col Annotator_2
Indexes of errors: [73, 99, 150, 446, 534, 650, 678, 1172, 1426, 1597, 1727]
Errors found in col Annotator_3
Indexes of errors: [194, 434, 635, 653, 726, 729, 962, 1106, 1254, 1536, 1669]


In [45]:
check_vals(unique_vals_exp, df, df.columns[1:])

no errors found


# Gold Standard

In [47]:
df['Gold_Standard'] = 42
for idx ,row in df[['Annotator_1','Annotator_2','Annotator_3']].iterrows():
  vals = np.array([row['Annotator_1'], row['Annotator_2'], row['Annotator_3']])
  df.loc[idx, 'Gold_Standard'] = np.argmax(np.bincount(vals))

In [48]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3,Gold_Standard
0,⚡Se non ci fossero soldati non ci sarebbero gu...,0,1,1,1
1,"21/08/21]( [PRE-PRINT]\n\n📄__ ""Shedding of Inf...",1,1,1,1
2,PAURA E DELIRIO ALLA CNN: IL MINISTERO DELLA V...,1,0,1,1
3,L'Aspirina non aumenta la sopravvivenza dei pa...,0,1,0,0
4,L'Italia non puo' dare armi lo vieta la Costit...,0,1,1,1


# Inter-annotator agreement

### **K = P(A) - P(E) / 1 - P(E)**

P(A) = Volte in cui gli annotatori si sono mostrati d'accordo

P(E) = Volte in cui ci si aspetta che gli annotatori per caso compiano la stessa scelta (e = expected)

The higher the better

In [49]:
def interannotator_agreement(annotator1, annotator2):
  num_records = len(annotator1)
  label1, label2 = np.unique(annotator1)

  counts1 = np.unique(annotator1, return_counts=True)[1]
  counts2 = np.unique(annotator2, return_counts=True)[1]
  annotation_prob = {
      'Annotator_1': {
          'label1': counts1[0]/num_records,
          'label2': counts1[1]/num_records
      },
      'Annotator_2': {
          'label1': counts2[0]/num_records,
          'label2': counts2[1]/num_records
      }
  }

  alignment = 0
  for val1, val2 in zip(annotator1, annotator2):
    if val1 == val2:
      alignment +=1

  p_a = alignment/num_records
  p_e = (annotation_prob['Annotator_1']['label1'] * \
        annotation_prob['Annotator_2']['label1']) + \
        (annotation_prob['Annotator_1']['label2'] * \
        annotation_prob['Annotator_2']['label2'])
  k =  (p_a - p_e) / (1 - p_e)

  return k



In [50]:
from statistics import harmonic_mean, mean

k1 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_2']))
k2 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_3']))
k3 = interannotator_agreement(np.array(df['Annotator_2']), np.array(df['Annotator_3']))
agg = (k1+k2+k3)/3

print('k1:', k1)
print('k2:', k2)
print('k3:', k3)
print('Mean:', agg)



k1: 0.9815402354955204
k2: 0.9804533299376035
k3: 0.9815395390131547
Mean: 0.9811777014820929
